# Data Cleaning
To clean this data set we'll start out by loading the dataset, checking for duplicates, and dropping columns that are not relevant to our analysis.

First, we'll load in our packages, set up our directories, and load in the dataset and take a look at it.

In [1]:
import pandas as pd
from pathlib import Path

#Set up directories
data_dir = Path('../data')
input_dir = data_dir / 'input'
output_dir = data_dir / 'output'

df = pd.read_csv(input_dir / 'phase2_sample.csv').set_index('Unnamed: 0')
df.index.names = ['Index']
df.head()

,indexed,reference-count,publisher,issue,license,content-domain,short-container-title,published-print,DOI,type,created,page,source,is-referenced-by-count,title,prefix,volume,author,member,reference,container-title,language,link,deposited,score,resource,issued,references-count,journal-issue,URL,ISSN,issn-type,subject,published,alternative-id,published-online,archive,update-policy,assertion,funder,article-number,accepted,abstract,original-title,subtitle,published-other,editor
Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,"{'date-parts': [[2022, 10, 7]], 'date-time': '...",14,Wiley,1,"[{'start': {'date-parts': [[2015, 9, 1]], 'dat...","{'domain': [], 'crossmark-restriction': False}",['Syst. Dyn. Rev.'],{'date-parts': [[2000]]},10.1002/(sici)1099-1727(200021)16:1<27::aid-sd...,journal-article,"{'date-parts': [[2002, 9, 10]], 'date-time': '...",27-41,Crossref,57,['The validation of commercial system dynamics...,10.1002,16,"[{'given': 'Geoff', 'family': 'Coyle', 'sequen...",311.0,[{'key': '10.1002/(SICI)1099-1727(200021)16:1<...,['System Dynamics Review'],en,[{'URL': 'https://api.wiley.com/onlinelibrary/...,"{'date-parts': [[2021, 7, 1]], 'date-time': '2...",0.0,{'primary': {'URL': 'https://onlinelibrary.wil...,{'date-parts': [[2000]]},14,"{'issue': '1', 'published-print': {'date-parts...",http://dx.doi.org/10.1002/(sici)1099-1727(2000...,"['0883-7066', '1099-1727']","[{'value': '0883-7066', 'type': 'print'}, {'va...","['Management of Technology and Innovation', 'S...",{'date-parts': [[2000]]},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'date-parts': [[2022, 3, 29]], 'date-time': '...",12,Springer Science and Business Media LLC,1,"[{'start': {'date-parts': [[1979, 3, 1]], 'dat...","{'domain': [], 'crossmark-restriction': False}",['MTB'],"{'date-parts': [[1979, 3]]}",10.1007/bf02653972,journal-article,"{'date-parts': [[2007, 7, 17]], 'date-time': '...",57-62,Crossref,20,['Effect of system geometry on the leaching be...,10.1007,10,"[{'given': 'C.', 'family': 'Vu', 'sequence': '...",297.0,"[{'key': 'BF02653972_CR1', 'volume-title': 'Ph...",['Metallurgical Transactions B'],en,[{'URL': 'http://link.springer.com/content/pdf...,"{'date-parts': [[2019, 5, 20]], 'date-time': '...",0.0,{'primary': {'URL': 'http://link.springer.com/...,"{'date-parts': [[1979, 3]]}",12,"{'issue': '1', 'published-print': {'date-parts...",http://dx.doi.org/10.1007/bf02653972,"['0360-2141', '1543-1916']","[{'value': '0360-2141', 'type': 'print'}, {'va...","['Materials Chemistry', 'Metals and Alloys', '...","{'date-parts': [[1979, 3]]}",['BF02653972'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'date-parts': [[2022, 3, 30]], 'date-time': '...",0,Wiley,3,"[{'start': {'date-parts': [[2017, 11, 1]], 'da...","{'domain': [], 'crossmark-restriction': False}",['RECIEL'],"{'date-parts': [[2017, 11]]}",10.1111/reel.12221,journal-article,"{'date-parts': [[2017, 12, 1]], 'date-time': '...",243-254,Crossref,2,['The international law on transboundary haze ...,10.1111,26,"[{'given': 'Shawkat', 'family': 'Alam', 'seque...",311.0,NaN,"['Review of European, Comparative &amp; Intern...",en,[{'URL': 'https://api.wiley.com/onlinelibrary/...,"{'date-parts': [[2017, 12, 1]], 'date-time': '...",0.0,{'primary': {'URL': 'http://doi.wiley.com/10.1...,"{'date-parts': [[2017, 11]]}",0,"{'issue': '3', 'published-print': {'date-parts...",http://dx.doi.org/10.1111/reel.12221,['2050-0386'],"[{'value': '2050-0386', 'type': 'print'}]","['Law', 'Management, Monitoring, Policy and La...","{'date-parts': [[2017, 11]]}",NaN,"{'date-parts': [[2017, 11, 28]]}",['Portico'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'date-parts': [[2022, 4, 3]], 'date-time': '2...",0,Crop Science Society of Japan,1-2,NaN,"{'domain': [], 'crossmark-restriction': False}","['Japanese journal of crop science', 'Jpn. J. ...",{'date-parts': [[1951]]},10.1626/jcs.20.219,journal-article,"{'date-parts': [[2011, 9, 20]], 'date-time': '...",219-222,Crossref,0,['Studies on the influence of pruning on the 

## Duplicate Records
Looking at the shape of the dataset against the number of unique DOIs will let us know just how many duplicate records we have.


In [2]:
df.shape

(100107, 47)

In [3]:
len(set(df['DOI']))

100044

In [4]:
#Dropping duplicate records
df.drop_duplicates(subset=['DOI'], keep='first', inplace=True)
df.shape

(100044, 47)

## Missing values and Irrelevant Columns
We're going to see just how many records are missing data and, considering the scope and objectives of our analysis, we'll drop some of the columns that we won't be using.

In [5]:
#Looking at null values by column
missing_values_count = df.isnull().sum()
missing_values_count

indexed                       0
reference-count               0
publisher                    22
issue                     11940
license                   46166
content-domain                0
short-container-title     10168
published-print           14778
DOI                           0
type                          0
created                       0
page                       9106
source                        0
is-referenced-by-count        0
title                       303
prefix                        0
volume                     4924
author                    10268
member                       22
reference                 47983
container-title               0
language                  20170
link                      16459
deposited                     0
score                         0
resource                      1
issued                        0
references-count              0
journal-issue             11922
URL                           0
ISSN                        338
issn-typ

Some of these columns with a high number of null values are of particular interest to us, such as the *abstract* column, so we'll be sure to keep that one. Most of the columns towards to end we won't be using so we'll drop them just to make our cleaned dataset a little less cumbersome. Additionally, there are other columns with few null values that we will drop for much the same reason, such as *score* and *ISSN*. We will also drop columns like *source* and *type* because all the records have the same *source: Crossref* and the same *type: journal-article*.

In [6]:
df.source.unique()

array(['Crossref'], dtype=object)

In [7]:
df.type.unique()

array(['journal-article'], dtype=object)

In [8]:
#Dropping some of the unnnecessary columns
#dropped_cols = ['indexed', 'funder', 'accepted', 'content-domain', 'article-number', 'archive', 'published-other', 'issue', 'type', 'ISSN', 'issn-type',
               #'reference-count', 'score', 'published-online', 'published-print', 'update-policy', 'source', 'page', 'license']
#df.drop(columns=dropped_cols, inplace=True)


## Will not be dropping columns
Select columns of interest when importing data. Maintain data sctructure in file.

## Editors
There are very few records that have a value in the *editor* column. Some of our prior work indicates that this can be a sign of a work that has been mislabeled as a 'journal article'. So we'll explore some of the records with a value in the editor column in order to verify that.

We'll set up a dataframe of just those records that have data in the *editor* column.

Next, we'll search the titles of these records for a few keywords.

In [9]:
editorial = df.loc[df.title.str.contains(r'editorial|errata|contents|conference|proceedings|masthead|symposium|abstract|Book Review', 
                                         regex=True, case=False, na=False)]
editorial

,indexed,reference-count,publisher,issue,license,content-domain,short-container-title,published-print,DOI,type,created,page,source,is-referenced-by-count,title,prefix,volume,author,member,reference,container-title,language,link,deposited,score,resource,issued,references-count,journal-issue,URL,ISSN,issn-type,subject,published,alternative-id,published-online,archive,update-policy,assertion,funder,article-number,accepted,abstract,original-title,subtitle,published-other,editor
Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,"{'date-parts': [[2022, 3, 31]], 'date-time': '...",1,Wiley,52,"[{'start': {'date-parts': [[2015, 9, 1]], 'dat...","{'domain': [], 'crossmark-restriction': False}",['ChemInform'],"{'date-parts': [[2015, 12]]}",10.1002/chin.201552196,journal-article,"{'date-parts': [[2015, 12, 13]], 'date-time': ...",no-no,Crossref,0,['ChemInform Abstract: Supramolecular Polymeri...,10.1002,46,"[{'given': 'Takeharu', 'family': 'Haino', 'seq...",311.0,"[{'key': '10.1002/chin.201552196-BIB1|cit1', '...",['ChemInform'],en,[{'URL': 'https://api.wiley.com/onlinelibrary/...,"{'date-parts': [[2021, 2, 17]], 'date-time': '...",0.0,{'primary': {'URL': 'http://doi.wiley.com/10.1...,"{'date-parts': [[2015, 12]]}",1,"{'issue': '52', 'published-print': {'date-part...",http://dx.doi.org/10.1002/chin.201552196,['0931-7597'],"[{'value': '0931-7597', 'type': 'print'}]",['General Materials Science'],"{'date-parts': [[2015, 12]]}",NaN,"{'date-parts': [[2015, 12, 10]]}",['Portico'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,"{'date-parts': [[2022, 4, 4]], 'date-time': '2...",0,Elsevier BV,1,"[{'start': {'date-parts': [[1965, 6, 1]], 'dat...","{'domain': [], 'crossmark-restriction': False}",['IFAC Proceedings Volumes'],"{'date-parts': [[1965, 6]]}",10.1016/s1474-6670(17)69139-0,journal-article,"{'date-parts': [[2017, 7, 1]], 'date-time': '2...",577,Crossref,0,['Symposium Closing Remarks'],10.1016,2,NaN,78.0,NaN,['IFAC Proceedings Volumes'],en,[{'URL': 'https://api.elsevier.com/content/art...,"{'date-parts': [[2018, 8, 30]], 'date-time': '...",0.0,{'primary': {'URL': 'https://linkinghub.elsevi...,"{'date-parts': [[1965, 6]]}",0,"{'issue': '1', 'published-print': {'date-parts...",http://dx.doi.org/10.1016/s1474-6670(17)69139-0,['1474-6670'],"[{'value': '1474-6670', 'type': 'print'}]","['General Economics, Econometrics and Finance']","{'date-parts': [[1965, 6]]}",['S1474667017691390'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,"{'date-parts': [[2022, 3, 29]], 'date-time': '...",0,Wiley,3,"[{'start': {'date-parts': [[2015, 9, 1]], 'dat...","{'domain': [], 'crossmark-restriction': False}",['Commun. Numer. Meth. Engng.'],"{'date-parts': [[1995, 3]]}",10.1002/cnm.1640110301,journal-article,"{'date-parts': [[2005, 8, 8]], 'date-time': '2...",fmi-fmi,Crossref,0,['Masthead'],10.1002,11,NaN,311.0,NaN,['Communications in Numerical Methods in Engin...,en,[{'URL': 'https://api.wiley.com/onlinelibrary/...,"{'date-parts': [[2021, 7, 2]], 'date-time': '2...",0.0,{'primary': {'URL': 'https://onlinelibrary.wil...,"{'date-parts': [[1995, 3]]}",0,"{'issue': '3', 'published-print': {'date-parts...",http://dx.doi.org/10.1002/cnm.1640110301,['1069-8299'],"[{'value': '1069-8299', 'type': 'print'}]","['Applied Mathematics', 'Computational Theory ...","{'date-parts': [[1995, 3]]}",NaN,"{'date-parts': [[2005, 6, 21]]}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,"{'date-parts': [[2022, 4, 2]], 'date-time': '2...",1,Wiley,33,"[{'start': {'date-parts': [[2015, 9, 1]], 'dat...","{'domain': [], 'crossmark-restriction': False}",['Chemischer Informationsdienst'],"{'date-parts': [[1972, 8, 15]]}",10.1002/chin.197233207,journal-article,"{'date-parts': [[2016, 2, 26]], 'date-time': '...",no-no,Crossref,0,['ChemInform Abstract: FRIEDEL-CRAFTS-ACYLIERU...,10.1002,3,"[{'given': 'J. K.', 'family': 'GROVES', 'seque...",311.0,"[{'key': '10.1002/chin.197233207-BIB1|cit1', '...",['Chemischer Informationsdienst'],en,[{'URL': 'http://api.wiley.com/onlinelibrary/t...,"{'date-pa

We've found some editorials, Mastheads, conference proceedings, and abstracts. We'll go ahead and drop them from our dataset.

In [10]:
df.drop(editorial.index, inplace=True)

In [11]:
df.shape

(96781, 47)

## Conferences
Looking back at **editorial** we see that there are a couple 'Conferences' and 'Proceedings' in the *container-title* column. Let's take a look at just how many records remain in our dataset are from these journals/containers.

Additionally, we see a few records from the journal *ChemInform*, a journal that published chemistry abstracts, we'll check to see if any of those records remain as well.

We'll use a keyword search in the *container-title* column to find these records.

In [12]:
conferences = df.loc[df['container-title'].str.contains(r'conference|ChemInform', regex=True, case=False)]
conferences

,indexed,reference-count,publisher,issue,license,content-domain,short-container-title,published-print,DOI,type,created,page,source,is-referenced-by-count,title,prefix,volume,author,member,reference,container-title,language,link,deposited,score,resource,issued,references-count,journal-issue,URL,ISSN,issn-type,subject,published,alternative-id,published-online,archive,update-policy,assertion,funder,article-number,accepted,abstract,original-title,subtitle,published-other,editor
Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
57,"{'date-parts': [[2022, 3, 31]], 'date-time': '...",6,IOP Publishing,1,"[{'start': {'date-parts': [[2021, 2, 1]], 'dat...","{'domain': ['iopscience.iop.org'], 'crossmark-...",['IOP Conf. Ser.: Earth Environ. Sci.'],"{'date-parts': [[2021, 2, 1]]}",10.1088/1755-1315/660/1/012131,journal-article,"{'date-parts': [[2021, 2, 22]], 'date-time': '...",012131,Crossref,0,['Application of wavelet multi-scale analysis ...,10.1088,660,"[{'given': 'Hailong', 'family': 'Sun', 'sequen...",266.0,"[{'key': 'EES_660_1_012131bib1', 'author': 'Ha...",['IOP Conference Series: Earth and Environment...,NaN,[{'URL': 'https://iopscience.iop.org/article/1...,"{'date-parts': [[2022, 1, 29]], 'date-time': '...",0.0,{'primary': {'URL': 'https://iopscience.iop.or...,"{'date-parts': [[2021, 2, 1]]}",6,"{'issue': '1', 'published-print': {'date-parts...",http://dx.doi.org/10.1088/1755-1315/660/1/012131,"['1755-1307', '1755-1315']","[{'value': '1755-1307', 'type': 'print'}, {'va...",NaN,"{'date-parts': [[2021, 2, 1]]}",NaN,NaN,NaN,http://dx.doi.org/10.1088/crossmark-policy,[{'value': 'Application of wavelet multi-scale...,NaN,NaN,NaN,<jats:title>Abstract</jats:title>\n ...,NaN,NaN,NaN,NaN
117,"{'date-parts': [[2022, 4, 2]], 'date-time': '2...",17,EDP Sciences,NaN,"[{'start': {'date-parts': [[2021, 4, 26]], 'da...","{'domain': [], 'crossmark-restriction': False}",['EPJ Web Conf.'],{'date-parts': [[2021]]},10.1051/epjconf/202124801022,journal-article,"{'date-parts': [[2021, 4, 26]], 'date-time': '...",01022,Crossref,0,['Distributions of Two Atoms Collisions over t...,10.1051,248,"[{'given': 'Sergey', 'family': 'Zheltov', 'seq...",250.0,"[{'key': 'R1', 'doi-asserted-by': 'crossref', ...",['EPJ Web of Conferences'],NaN,[{'URL': 'https://www.epj-conferences.org/10.1...,"{'date-parts': [[2021, 4, 26]], 'date-time': '...",0.0,{'primary': {'URL': 'https://www.epj-conferenc...,{'date-parts': [[2021]]},17,NaN,http://dx.doi.org/10.1051/epjconf/202124801022,['2100-014X'],"[{'value': '2100-014X', 'type': 'electronic'}]","['General Earth and Planetary Sciences', 'Gene...",{'date-parts': [[2021]]},['epjconf_mnps2021_01022'],"{'date-parts': [[2021, 4, 26]]}",NaN,NaN,NaN,NaN,NaN,NaN,<jats:p>The processes of heat and mass transfe...,NaN,NaN,NaN,"[{'given': 'A.', 'family': 'Nadykto', 'sequenc..."
140,"{'date-parts': [[2022, 11, 8]], 'date-time': '...",0,Association for the Advancement of Artificial ...,1,NaN,"{'domain': [], 'crossmark-restriction': False}",['AAAI'],NaN,10.1609/aaai.v32i1.11721,journal-article,"{'date-parts': [[2022, 6, 24]], 'date-time': '...",NaN,Crossref,6,['SC2Net: Sparse LSTMs for Sparse Coding'],10.1609,32,"[{'given': 'Joey Tianyi', 'family': 'Zhou', 's...",9382.0,NaN,['Proceedings of the AAAI Conference on Artifi...,NaN,[{'URL': 'https://ojs.aaai.org/index.php/AAAI/...,"{'date-parts': [[2022, 11, 7]], 'date-time': '...",0.0,{'primary': {'URL': 'https://ojs.aaai.org/inde...,"{'date-parts': [[2018, 4, 29]]}",0,"{'issue': '1', 'published-online': {'date-part...",http://dx.doi.org/10.1609/aaai.v32i1.11721,"['2374-3468', '2159-5399']","[{'value': '2374-3468', 'type': 'electronic'},...",['General Medicine'],"{'date-parts': [[2018, 4, 29]]}",NaN,"{'date-parts': [[2018, 4, 29]]}",NaN,NaN,NaN,NaN,NaN,NaN,<jats:p>\n \n The iterative hard-t...,NaN,NaN,NaN,NaN
173,"{'date-parts': [[2022, 4, 1]], 'date-time': '2...",3,Springer Science and Business Media LLC,S1,"[{'start': {'date-parts': [[2013, 3, 1]], 'dat...","{'domain': ['link.springer.com']

In [13]:
df.drop(conferences.index, inplace=True)

## Cleaning Dates
Here we are going to re-format some of the datetime columns into a more easily parsed format. *Created, deposited* and *published*. Not all records have month and day values for the *published* field, so we'll only take the year from those. For *created* and *deposited* we will have a YYYY-MM-DD format.

We've chosen these dates because they reflect certain information that we'll use later on. *Created* is the date when the item was first inserted into the Crossref database. *Deposited* reflects the last time the record was entered by the publisher (potentially with changes to the record but necessarily the case). *Published* reflects when the item itself was actually published.

We'll use a regular expression to extract the dates from each of the records in each of those three columns, then we'll convert them to datetime dtypes.

In [14]:
date_columns = ['created', 'deposited']

for col in date_columns:
    df[col] = df[col].str.extract(r"\'(\d{4}\S\d{2}\S\d{2})")
    df[col] = pd.to_datetime(df[col], format="%Y-%m-%d")
df['published'] = df['published'].str.extract(r"(\d{4})")
df.head()

,indexed,reference-count,publisher,issue,license,content-domain,short-container-title,published-print,DOI,type,created,page,source,is-referenced-by-count,title,prefix,volume,author,member,reference,container-title,language,link,deposited,score,resource,issued,references-count,journal-issue,URL,ISSN,issn-type,subject,published,alternative-id,published-online,archive,update-policy,assertion,funder,article-number,accepted,abstract,original-title,subtitle,published-other,editor
Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,"{'date-parts': [[2022, 10, 7]], 'date-time': '...",14,Wiley,1,"[{'start': {'date-parts': [[2015, 9, 1]], 'dat...","{'domain': [], 'crossmark-restriction': False}",['Syst. Dyn. Rev.'],{'date-parts': [[2000]]},10.1002/(sici)1099-1727(200021)16:1<27::aid-sd...,journal-article,2002-09-10,27-41,Crossref,57,['The validation of commercial system dynamics...,10.1002,16,"[{'given': 'Geoff', 'family': 'Coyle', 'sequen...",311.0,[{'key': '10.1002/(SICI)1099-1727(200021)16:1<...,['System Dynamics Review'],en,[{'URL': 'https://api.wiley.com/onlinelibrary/...,2021-07-01,0.0,{'primary': {'URL': 'https://onlinelibrary.wil...,{'date-parts': [[2000]]},14,"{'issue': '1', 'published-print': {'date-parts...",http://dx.doi.org/10.1002/(sici)1099-1727(2000...,"['0883-7066', '1099-1727']","[{'value': '0883-7066', 'type': 'print'}, {'va...","['Management of Technology and Innovation', 'S...",2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'date-parts': [[2022, 3, 29]], 'date-time': '...",12,Springer Science and Business Media LLC,1,"[{'start': {'date-parts': [[1979, 3, 1]], 'dat...","{'domain': [], 'crossmark-restriction': False}",['MTB'],"{'date-parts': [[1979, 3]]}",10.1007/bf02653972,journal-article,2007-07-17,57-62,Crossref,20,['Effect of system geometry on the leaching be...,10.1007,10,"[{'given': 'C.', 'family': 'Vu', 'sequence': '...",297.0,"[{'key': 'BF02653972_CR1', 'volume-title': 'Ph...",['Metallurgical Transactions B'],en,[{'URL': 'http://link.springer.com/content/pdf...,2019-05-20,0.0,{'primary': {'URL': 'http://link.springer.com/...,"{'date-parts': [[1979, 3]]}",12,"{'issue': '1', 'published-print': {'date-parts...",http://dx.doi.org/10.1007/bf02653972,"['0360-2141', '1543-1916']","[{'value': '0360-2141', 'type': 'print'}, {'va...","['Materials Chemistry', 'Metals and Alloys', '...",1979,['BF02653972'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'date-parts': [[2022, 3, 30]], 'date-time': '...",0,Wiley,3,"[{'start': {'date-parts': [[2017, 11, 1]], 'da...","{'domain': [], 'crossmark-restriction': False}",['RECIEL'],"{'date-parts': [[2017, 11]]}",10.1111/reel.12221,journal-article,2017-12-01,243-254,Crossref,2,['The international law on transboundary haze ...,10.1111,26,"[{'given': 'Shawkat', 'family': 'Alam', 'seque...",311.0,NaN,"['Review of European, Comparative &amp; Intern...",en,[{'URL': 'https://api.wiley.com/onlinelibrary/...,2017-12-01,0.0,{'primary': {'URL': 'http://doi.wiley.com/10.1...,"{'date-parts': [[2017, 11]]}",0,"{'issue': '3', 'published-print': {'date-parts...",http://dx.doi.org/10.1111/reel.12221,['2050-0386'],"[{'value': '2050-0386', 'type': 'print'}]","['Law', 'Management, Monitoring, Policy and La...",2017,NaN,"{'date-parts': [[2017, 11, 28]]}",['Portico'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'date-parts': [[2022, 4, 3]], 'date-time': '2...",0,Crop Science Society of Japan,1-2,NaN,"{'domain': [], 'crossmark-restriction': False}","['Japanese journal of crop science', 'Jpn. J. ...",{'date-parts': [[1951]]},10.1626/jcs.20.219,journal-article,2011-09-20,219-222,Crossref,0,['Studies on the influence of pruning on the v...,10.1626,20,"[{'given': 'C.', 'family': 'TSUDA', 'sequence'...",632.0,NaN,['Japanese Journal of Crop Science'],en,[{'URL': 'http://www.jstage.jst.go.jp/article/...,2021-04-30,0.0,{'primary': {'URL': 'http://www.jstage.jst.go....,{'date-parts': [[1951]]},0,"{'issue': '1-2', 'published-print': {'date-par...",http://dx.doi.org/10.1626/jcs.20.219,"['0011-1

# String slicing
Now that the dates are converted, one of the last problems to address are some of the excess character in the *title, short-container-title,* and *container-title* fields.

In [15]:
cols = ['title', 'short-container-title', 'container-title']
for col in cols:
    df[col] = df[col].str.slice(start=2, stop=-2)

In [16]:
df['title'][0]

'The validation of commercial system dynamics models'

## Cleaning XML tags
We'll be looking at the abstract column, so it will benefit us to clean out the tags and only have te relevant text for each record. We'll write a quick function to do that.

In [17]:
#import beatuiful soup
from bs4 import BeautifulSoup as bs
def clean_abstracts(abstract):
    try:
        soup = bs(abstract, features='lxml')
        stripped_strings = soup.get_text()
        return stripped_strings
    except:
        return None
stripped_abstracts = df.abstract.map(lambda x: clean_abstracts(x))
df['abstract'] = stripped_abstracts

In [18]:
df['abstract'][11]

'A novel and efficient approach to the synthesis of 2-vinylbenzylamines is reported. This involves obtaining 2-vinylbenzylamine ligands from tetrahydroisoquinoline by alkylation and reduction followed by the Hofmann cleavage. The resultant 2-vinylbenzylamines allowed us to obtain new Hoveyda–Grubbs catalysts, which were thoroughly characterised by NMR, ESIMS, and X-ray crystallography. The utility of this chemistry is further demonstrated by the tests of the novel catalysts (up to 10−2 mol %) in different metathesis reactions such as cross metathesis (CM), ring-closing metathesis (RCM) and ring-opening cross metathesis (ROCM).'

Looks great! Now we'll save our cleaned dataset.

In [19]:
df.shape

(95981, 47)

In [20]:
df.to_csv(input_dir / '02_cleaned_data.csv')